In [1]:
import pandas as pd
import numpy as np
import re

def load_dataframe(relative_path,dataframe_name):
    df = pd.read_pickle(f'{relative_path}/{dataframe_name}.pkl')    
    return df

def read_file(relative_path,file_name):
    text= ""
    with open(f'{relative_path}/{file_name}.abc','r') as f:
        text = f.read()
    return text

In [2]:
relative_path ="notebooks/data/final_dataset"
filename_name = 'clean_augmented_data'
#filename_name = 'clean_original_training_data'
#relative_path ="notebooks/data/original_dataset"
training_data_df = load_dataframe(relative_path,filename_name)
training_data_df.columns


Index(['unit_note_length', 'tuneBook', 'title', 'reference_number',
       'original_header', 'original_body', 'meter', 'key', 'clean_song',
       'clean_header', 'clean_body', 'chord_progression', '"fm"', '"ff'"',
       '"f7"', '"em"', '"ee'"', '"e7"', '"e"', '"dm"', '"dd'"', '"d7"', '"d"',
       '"cm"', '"cc'"', '"c7"', '"c#m"', '"c#7"', '"c"', '"Gm"', '"Gg"',
       '"Gd'"', '"G7"', '"G#m"', '"G#7"', '"G"', '"Fm"', '"Ff"', '"Fc'"',
       '"F7"', '"F#m"', '"F#7"', '"F"', '"Em"', '"Eb"', '"E7"', '"E#m"',
       '"E#7"', '"E"', '"Dm"', '"Da"', '"D7"', '"D#m"', '"D#7"', '"D"', '"Cm"',
       '"Cg"', '"C7"', '"C#m"', '"C#7"', '"C"', '"Bm"', '"Bf"', '"Bb"', '"B7"',
       '"B#m"', '"B#7"', '"B"', '"Am"', '"Ae'"', '"Aa"', '"A7"', '"A#m"',
       '"A#7"', '"A"'],
      dtype='object')

In [3]:
training_data_df.tail()

,unit_note_length,tuneBook,title,reference_number,original_header,original_body,meter,key,clean_song,clean_header,...,"""B#m""","""B#7""","""B""","""Am""","""Ae'""","""Aa""","""A7""","""A#m""","""A#7""","""A"""
9491,1/4,dataset_min5.abc,Grandpa's,78,X:78\nT:Grandpa's\nM:4/4\nL:1/4\nK:Amajor,"E/2D/2|""A,""CE""E7""FG|""A,""A/2G/2A/2B/2ce|""B,m""dc...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""|""Bm""""B7""|""E7""|...","M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""|""Bm""""B7""|""E7""|...",...,0,0,0,0,0,0,0,0,0,9
9492,1/4,dataset_min5.abc,The Girl With The Green Hat On,79,X:79\nT:The Girl With The Green Hat On\nM:4/4\...,"(3E/2F/2G/2|""A,""AE""E7""E/2F/2E/2D/2|""A,""C/2D/2E...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""""E7""|""A""|""E7""|""...","M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""""E7""|""A""|""E7""|""...",...,0,0,0,0,0,0,0,0,0,9
9493,1/4,dataset_min5.abc,Green Meadow,80,X:80\nT:Green Meadow\nM:4/4\nL:1/4\nK:Dmajor,"(3A,/2B,/2C/2|""D""DD/2E/2F/2D/2F/2A/2|""G,""B/2c/...",4/4,D,"M:4/4\nL:1/4\nK:D\n|""D""|""G""""D""|""Em""|""Em""""A7""|""...","M:4/4\nL:1/4\nK:D\n|""D""|""G""""D""|""Em""|""Em""""A7""|""...",...,0,0,0,0,0,0,7,0,0,0
9494,1/4,dataset_min5.abc,The Old Grey Cat,82,X:82\nT:The Old Grey Cat\nM:4/4\nL:1/4\nK:Bminor,"F|""B,m""BBB,B,/2C/2|""B,m""D/2C/2D/2E/2F/2E/2F/2^...",4/4,Bm,"M:4/4\nL:1/4\nK:Bm\n|""Bm""|""Bm""|""A""|""A""|""Bm""|""B...","M:4/4\nL:1/4\nK:Bm\n|""Bm""|""Bm""|""A""|""A""|""Bm""|""B...",...,0,0,0,0,0,0,0,0,0,5
9495,1/4,dataset_min5.abc,Gyre And Gimble,84,X:84\nT:Gyre And Gimble\nM:4/4\nL:1/4\nK:Amajor,"E|""A,""AECE|""B,m""FD""E7""B,D|""A,""CEA3/2B/2|""E7""c/...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""|""Bm""""E7""|""A""|""E7""""A""|""...","M:4/4\nL:1/4\nK:A\n|""A""|""Bm""""E7""|""A""|""E7""""A""|""...",...,0,0,0,0,0,0,0,0,0,12


In [6]:
training_data_df["clean_header"].str.len().mean()

124.49420808761583

In [7]:
training_data_df["clean_body"].str.len().mean()

301.346251053075

In [8]:
bodies = ""
silences = 0
for body in training_data_df["clean_body"]:
    if 'z' in body:
        silences +=1 
    bodies += body+"\n"
chars = sorted(list(set(bodies)))
vocab_size = len(chars)
print('vocab: ',''.join(chars))
print('vocab_size',vocab_size)
print("silences ",silences)

vocab:  
"#'(),-/1234567=ABCDEFG[]^_abcdefgmz|~
vocab_size 39
silences  516


In [9]:
training_data_text = read_file(relative_path,filename_name)

print("number of chars:",len(training_data_text))

number of chars: 4062773


In [10]:
chars = sorted(list(set(training_data_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


"#'(),-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
45


In [11]:
import torch

print(torch.__version__)

2.0.0+cu117


In [12]:
import transformers
print(transformers.__version__)

4.28.0.dev0


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import wandb
import tiktoken

print(wandb.__version__)

0.14.2


In [14]:
!ls

Dockerfile  docker-compose.yaml  overrides.json
README.md   notebooks		 requirements.txt


In [15]:
from dotenv import load_dotenv
import os
nano_path = 'notebooks/nanoGPT'
os.chdir(nano_path)
load_dotenv()

True

In [16]:
!ls

LICENSE      assets	      data	  out-abc-char	wandb
README.md    config	      model.py	  sample.py
__pycache__  configurator.py  older_ckpt  train.py


In [ ]:
# Dataset with multiple voices present
#length of dataset in characters: 4,149,703
#all the unique characters: 
#"#'()+,-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
#vocab size: 46
#train has 3,734,732 tokens
#val has 414,971 tokens

In [17]:
!python3 data/abc_char/prepare.py

length of dataset in characters: 4,062,773
all the unique characters: 
"#'(),-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
vocab size: 45
train has 3,656,495 tokens
val has 406,278 tokens


In [18]:
!python3 train.py config/train_abc_char.py

Overriding config with config/train_abc_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-abc-char'
eval_interval = 10 # keep frequent because we'll overfit
eval_iters = 500
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
wandb_project = 'abc-char'
wandb_run_name = 'mini-char-gpt-hd-8-ly-12-bt-4-ctx-1024'

dataset = 'abc_char'
batch_size = 4
block_size = 1024 # context of up to 512 previous characters

# baby GPT model :)
n_layer = 12
n_head = 8
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_ite

In [29]:
folder_name = 'hd-8-ly-12-bt-4-ctx-1024/'
examples_folder = f'./older_ckpt/{folder_name}'
examples_folder

'./older_ckpt/hd-8-ly-12-bt-4-ctx-1024/'

In [ ]:
common_roman_num_prog = '|"I"|"I"|"IV"|"V"|"V"|"I"|"IV"|"V"|"I"|'
G_roman_chord_progression = f''

songs_start = {
    'G' :'M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]',
    'C' :'M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]',
    'Am':'M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]'
    }

songs_roman_start = {
    'G' :'M:4/4L:1/4K:G|"I"|"IV"|"V"|"V"|"I"|"IV"|"V"|"I"|]',
    'C' :'M:4/4L:1/4K:C|"I"|"IV"|"V"|"V"|"I"|"IV"|"V"|"I"|]',
    'Am':'M:4/4L:1/4K:Am|"i"|"iv"|"V"|"V"|"i"|"iv"|"V"|"i"|]'
}

## Test key with most occurrences: G

In [30]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]' > {examples_folder}/examples_G.txt

## Test major key with low samples: C

In [26]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]' > {examples_folder}/examples_C.txt

## Test minor key with low samples: Am

In [27]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]' > {examples_folder}/examples_Am.txt

## Move checkpoint files

In [32]:
source = './data/abc_char/meta.pkl'
target_folder = examples_folder
!mv {source} {target_folder}/meta.pkl

In [37]:
source_folder = './out-abc-char/ckpt.pt'
!mv {source} {target_folder}/ckpt.pt

In [36]:
source = './config/train_abc_char.py'
!cp {source} {target_folder}/config.txt

## Test older checkpoint

In [ ]:
 !python3 sample.py --out_dir=older_ckpt/m_voices --path_meta=older_ckpt/m_voices --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

In [ ]:
cat older_ckpt/m_voices/ckpt.pkl

In [ ]:
!ls -l older_ckpt/m_voices

In [ ]:
!ls -l out-abc-char/

In [ ]:
!id